<a href="https://colab.research.google.com/github/alwaysY0ung/hg-mldl/blob/main/ch05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

테스트세트로 일반화 성능을 올바르게 예측하며면 가능한 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋다. 이를 위해서 데이터를 어떻게 분류해야할까? 간단하다. train_test_split 메서드를 두 번 호출하는 방법이다,

# 데이터

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
# 0 <- Red Wine
# 1 <- White Wine

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# train_test_split

이번엔 valid도 준비하자

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


훈련세트에 대한 정확도가 굉장히 높다. 훈련세트에 대해 굉장히 과대적합된 모델이다.

# 교차검증

valid를 만드느라 훈련세트가 적어진 문제를 해결하기 위한 방법이다. 보통은 5-폴드 교차검증이나, 10-폴드 교차검증을 많이 사용한다.


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

import numpy as np
print(np.mean(scores['test_score']))

{'fit_time': array([0.01122427, 0.01047134, 0.0113306 , 0.0107348 , 0.01438832]), 'score_time': array([0.00154066, 0.00149846, 0.00166941, 0.00208044, 0.00164032]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


# 하이퍼파라미터 튜닝, 그리드 서치

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

print(gs.best_params_)

print(gs.cv_results_['mean_test_score'])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
import numpy as np

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


# 랜덤서치 (넘빨라서 못함...)